In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import re

### cargar archivo 1

In [12]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padon_1)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/1ER TANDA ROCIO (SOLO TITULARES) EPV (CUENTA NUEVA).xlsx' mode='r' encoding='cp1252'>
      CUIL TITULAR*       CUIL*  APELLIDO*            NOMBRES*  \
0       27417656249         NaN     GALVAN            MICAELA    
1       27242534110         NaN     SORIA      ANDREA FERNANDA   
2       27317727238         NaN    BENITEZ  GABRIELA FERNANDA    
3       27358318555         NaN     SUAREZ              IVVANA   
4       20431778824         NaN     PEREZ             RODRIGO    
...             ...         ...        ...                 ...   
1721  20-23853224-9         NaN    FERRAUR      DANIEL ANTONIO   
1722  20-32341107-8         NaN  RODRIGUEZ         JUAN CARLOS   
1723  20-16258571-2         NaN   NICOLINI    ESTEBAN ENRRIQUE   
1724            NaN  49259513.0   NICOLINI       JAZMIN LUJAN    
1725    20313079709         NaN    ARRAYA        PABLO AGUSTIN   

         DOMICILIO (calle)* Domicilio (altura)*       LOCALIDAD*    TEL

### cargar archivo 2

In [20]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron_2 = pd.read_excel(ruta_archivo, engine='xlrd')
            padron_2 = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron_2)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/padron_completo_13-06-23.csv' mode='r' encoding='cp1252'>
       Nro. OS      Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7            Desregulado     SI             26407599   
1            7            Desregulado     SI             26407599   
2            6               Afiliado     SI             30694069   
3           10            Desregulado     SI             31403090   
4            1  Monotributista social     NO             36821087   
...        ...                    ...    ...                  ...   
15296        4            Desregulado     SI             21601831   
15297        4            Desregulado     SI             21601831   
15298        4            Desregulado     SI             21601831   
15299        4            Desregulado     SI             21601831   
15300        1               Afiliado     SI             36602782   

         Nro. Afiliado Parentesco con el TITULAR Apellido Afili

### Normalizar DNI archivo convenio

In [9]:

#armar el camp DNI juntando las los campos cuil titular y cuil familiar
padon_1['dni']=padon_1['CUIL TITULAR*'].fillna(padon_1['CUIL*'])
#limpiar el nuevo campo dni dejando solo los numeros 
padon_1['dni']=padon_1['dni'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
#dejar solo los numreros eliminar caracteres 
padon_1['dni2']=padon_1['dni'].apply(lambda x: x[2:10] if len(x)>= 8 else '0')
#extraer el dni del cuil y dejar vacios los q son menores  a 8
padon_1['dni2']=padon_1['dni2'].astype(int)
#pasar el campo a int 
padon_1['FECHA DE NAC.*']=padon_1['FECHA DE NAC.*'].astype(str)
#convertir fecha. 
print(padon_1['FECHA DE NAC.*'])
padon_1=funciones.transformar_fecha(padon_1, 'FECHA DE NAC.*')
#print(padon_1[['FECHA DE ALTA*','FECHA DE NAC.*','dni2']].head(30))


c:\Users\zickd\Documents\REPOS_GIT\osoetsylra\funciones.py:32: UserWarning: Parsing dates in MM/DD/YYYY format when dayfirst=True was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df[columna]=pd.to_datetime(df[columna], dayfirst=True, errors='coerce')
c:\Users\zickd\Documents\REPOS_GIT\osoetsylra\funciones.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].dt.strftime("%d/%m/%Y")


In [10]:
print(padon_1['FECHA DE NAC.*'])
padon_1.to_excel(r'C:\Users\zickd\Downloads\fechas.xlsx', engine='openpyxl', index=False)

0       12/02/1999
1       03/11/1974
2       19/09/1985
3       20/03/1991
4       03/02/2001
           ...    
1586    07/04/2002
1587    13/11/1999
1588    05/09/1977
1589    17/05/1974
1590    04/08/2004
Name: FECHA DE NAC.*, Length: 1544, dtype: object


### Merge archivos x DNI

In [44]:
cruce_tis=pd.merge(padon_1, padron_2, left_on='dni2', right_on='Nro. Documento', how='left', indicator=True)
#cruce_tis.to_excel(r'C:\Users\zickd\Downloads\merge.xlsx', engine='openpyxl', index=False)

cruce_tis_filtro=cruce_tis[cruce_tis['_merge']=='left_only']

#print(cruce_tis_filtro.head(5))

### Normalizar Fecha

### Errores

SyntaxError: invalid syntax (2464904157.py, line 1)

In [48]:
print(tis_errores['error1'])

TypeError: list indices must be integers or slices, not str

### exportar el cruce

In [46]:
save_path = filedialog.askdirectory()
#uso iloc por si las columnas tienen el mismo nombre, necesito la primeras 20 columnas
cruce_tis_filtro.iloc[:, :18].to_excel(save_path + '\cruce_tis.xlsx', engine='openpyxl' ,index = False )

